In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

# from googletrans import Translator
# translator = Translator(service_urls=[
#       'translate.google.com',
#       'translate.google.com.br',
#     ])

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

# 1. Explore Dataset

## Customer Dataset

In [39]:
customers = pd.read_csv('./Datasets/olist_customers_dataset.csv', dtype={'customer_zip_code_prefix': str})

In [40]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [41]:
customers.describe()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
count,99441,99441,99441,99441,99441
unique,99441,96096,14994,4119,27
top,89b009b9871c7df3d23c56b673ddef57,8d50f5eadf50201ccdcedfb9e2ac8455,22790,sao paulo,SP
freq,1,17,142,15540,41746


In [42]:
customers_df_summary = pd.DataFrame({
    'columns': customers.columns,
    'null_value_total': [customers[i].isnull().sum() for i in customers.columns],
    'unique_value_total': [customers[i].nunique() for i in customers.columns],
    'length': [customers[i].shape[0] for i in customers.columns],
    'data_type': [customers[i].dtypes for i in customers.columns]
})
customers_df_summary['dataset'] = 'customers'
customers_df_summary['null_value_total_pct'] = round(customers_df_summary['null_value_total'] / customers_df_summary['length'] * 100, 2)
customers_df_summary= customers_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
customers_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,customers,customer_id,0,0.0,99441,99441,object
1,customers,customer_unique_id,0,0.0,96096,99441,object
2,customers,customer_zip_code_prefix,0,0.0,14994,99441,object
3,customers,customer_city,0,0.0,4119,99441,object
4,customers,customer_state,0,0.0,27,99441,object


In [43]:
customers.duplicated().sum()

0

In [64]:
customers.customer_unique_id.nunique()

96096

<div class="alert alert-block alert-success">
Kolom yang akan dianalisis pada dataset customers berjumlah 5 kolom. Total customers (unique) berjumlah 96096 data. Jumlah data sebanyak 99441 data menandakan bahwa setiap customer_id bisa jadi memiliki customer unique id, kode pos, city dan state yang sama.
</div>

## Geolocation Dataset

In [65]:
geolocation = pd.read_csv('./Datasets/olist_geolocation_dataset.csv', dtype={'geolocation_zip_code_prefix': str})
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP


In [66]:
def change(x):
    temp = ''
    for i in x:
        if i == 'á' or i == 'ã' or i == 'â':
            temp += 'a'
        elif i == 'é' or i == 'ê':
            temp += 'e'
        elif i == 'ó' or i == 'ô' or i == 'õ':
            temp += 'o'
        elif i == 'ç':
            temp += 'c'
        elif i == 'í':
            temp += 'i'
        elif i == 'ú':
            temp += 'u'
        else:
            temp += i
    return temp
            
geolocation['geolocation_city'] = geolocation['geolocation_city'].map(lambda x: change(x))

In [67]:
geolocation.describe()

,geolocation_lat,geolocation_lng
count,1.000163e+06,1.000163e+06
mean,-2.117615e+01,-4.639054e+01
std,5.715866e+00,4.269748e+00
min,-3.660537e+01,-1.014668e+02
25%,-2.360355e+01,-4.857317e+01
50%,-2.291938e+01,-4.663788e+01
75%,-1.997962e+01,-4.376771e+01
max,4.506593e+01,1.211054e+02


In [47]:
temp_geo = geolocation.pivot_table(index='geolocation_zip_code_prefix', values=['geolocation_lat', 'geolocation_lng'], aggfunc='mean').reset_index()
temp_geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,01001,-23.550190,-46.634024
1,01002,-23.548146,-46.634979
2,01003,-23.548994,-46.635731
3,01004,-23.549799,-46.634757
4,01005,-23.549456,-46.636733


In [48]:
# Pada geolocation_zip_code_prefix yang sama, 
# geolocation_lat dan geolocation_lng memiliki nilai yang berbeda
geolocation[geolocation['geolocation_zip_code_prefix'] == '01029'].head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
8,01029,-23.543769,-46.634278,sao paulo,SP
78,01029,-23.541306,-46.632853,sao paulo,SP
128,01029,-23.537314,-46.633030,sao paulo,SP
232,01029,-23.540563,-46.632850,sao paulo,SP
295,01029,-23.543769,-46.634278,sao paulo,SP


In [49]:
geolocation_df_summary = pd.DataFrame({
    'columns': geolocation.columns,
    'null_value_total': [geolocation[i].isnull().sum() for i in geolocation.columns],
    'unique_value_total': [geolocation[i].nunique() for i in geolocation.columns],
    'length': [geolocation[i].shape[0] for i in geolocation.columns],
    'data_type': [geolocation[i].dtypes for i in geolocation.columns]
})
geolocation_df_summary['dataset'] = 'geolocation'
geolocation_df_summary['null_value_total_pct'] = round(geolocation_df_summary['null_value_total'] / geolocation_df_summary['length'] * 100, 2)
geolocation_df_summary= geolocation_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
geolocation_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,geolocation,geolocation_zip_code_prefix,0,0.0,19015,1000163,object
1,geolocation,geolocation_lat,0,0.0,717358,1000163,float64
2,geolocation,geolocation_lng,0,0.0,717613,1000163,float64
3,geolocation,geolocation_city,0,0.0,5971,1000163,object
4,geolocation,geolocation_state,0,0.0,27,1000163,object


<div class="alert alert-block alert-success">
Dataset geolocation memiliki 5 kolom. Total state dan city (unique) berjumlah masing-masing 27 dan 5971 data. Setiap geolocation_zip_code_prefix memiliki longitude dan latitude yang berbeda-beda. Longitude dan latitude pada masing-masing pelanggan tidak dapat ditentukan dalam dataset ini.
</div>

## Order Items

In [69]:
order_items = pd.read_csv('./Datasets/olist_order_items_dataset.csv', parse_dates=['shipping_limit_date'])
order_items.drop(columns=['shipping_limit_date'], inplace=True)
order_items.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [70]:
order_items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


In [71]:
orderitems_df_summary = pd.DataFrame({
    'columns': order_items.columns,
    'null_value_total': [order_items[i].isnull().sum() for i in order_items.columns],
    'unique_value_total': [order_items[i].nunique() for i in order_items.columns],
    'length': [order_items[i].shape[0] for i in order_items.columns],
    'data_type': [order_items[i].dtypes for i in order_items.columns]
})
orderitems_df_summary['dataset'] = 'order_items'
orderitems_df_summary['null_value_total_pct'] = round(orderitems_df_summary['null_value_total'] / orderitems_df_summary['length'] * 100, 2)
orderitems_df_summary= orderitems_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
orderitems_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,order_items,order_id,0,0.0,98666,112650,object
1,order_items,order_item_id,0,0.0,21,112650,int64
2,order_items,product_id,0,0.0,32951,112650,object
3,order_items,seller_id,0,0.0,3095,112650,object
4,order_items,price,0,0.0,5968,112650,float64
5,order_items,freight_value,0,0.0,6999,112650,float64


<div class="alert alert-block alert-success">
<b>Insight:</b> Dataset order_items memiliki 6 fitur. Total order (unique) berjumlah 98666 dengan data product yang diorder berjumlah 32951.
</div>

In [72]:
# Contoh customer yang membeli 12 item dalam sekali order
order_items[order_items['order_id'] == '1b15974a0141d54e36626dca3fdc731a']

,order_id,order_item_id,product_id,seller_id,price,freight_value
11932,1b15974a0141d54e36626dca3fdc731a,1,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11933,1b15974a0141d54e36626dca3fdc731a,2,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11934,1b15974a0141d54e36626dca3fdc731a,3,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11935,1b15974a0141d54e36626dca3fdc731a,4,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11936,1b15974a0141d54e36626dca3fdc731a,5,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11937,1b15974a0141d54e36626dca3fdc731a,6,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11938,1b15974a0141d54e36626dca3fdc731a,7,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11939,1b15974a0141d54e36626dca3fdc731a,8,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11940,1b15974a0141d54e36626dca3fdc731a,9,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
11941,1b15974a0141d54e36626dca3fdc731a,10,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12


In [73]:
# Contoh customer yang membeli 3 item dalam sekali order
order_items[order_items['order_id'] == '00143d0f86d6fbd9f9b38ab440ac16f5']

,order_id,order_item_id,product_id,seller_id,price,freight_value
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,21.33,15.1
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,21.33,15.1
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,21.33,15.1


<div class="alert alert-block alert-success">
Ada pelanggan yang sekali transaksi membeli > 1 produk namun tidak melakukan pembelian selanjutnya di kemudian hari, ada juga pelanggan yang membeli 1 produk saja namun melakukan pembelian selanjutnya di kemudian hari.
</div>

## Order Reviews

In [74]:
order_reviews = pd.read_csv('./Datasets/olist_order_reviews_dataset.csv')

order_reviews.head(10)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
5,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,NaN,NaN,2018-04-13 00:00:00,2018-04-16 00:39:37
6,07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,NaN,NaN,2017-07-16 00:00:00,2017-07-18 19:30:34
7,7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,NaN,NaN,2018-08-14 00:00:00,2018-08-14 21:36:06
8,a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,NaN,NaN,2017-05-17 00:00:00,2017-05-18 12:05:37
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47


In [75]:
order_reviews.shape

(100000, 7)

In [76]:
order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                100000 non-null  object
 1   order_id                 100000 non-null  object
 2   review_score             100000 non-null  int64 
 3   review_comment_title     11715 non-null   object
 4   review_comment_message   41753 non-null   object
 5   review_creation_date     100000 non-null  object
 6   review_answer_timestamp  100000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [77]:
order_reviews.describe()

,review_score
count,100000.000000
mean,4.070890
std,1.359663
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [78]:
orderreviews_df_summary = pd.DataFrame({
    'columns': order_reviews.columns,
    'null_value_total': [order_reviews[i].isnull().sum() for i in order_reviews.columns],
    'unique_value_total': [order_reviews[i].nunique() for i in order_reviews.columns],
    'length': [order_reviews[i].shape[0] for i in order_reviews.columns],
    'data_type': [order_reviews[i].dtypes for i in order_reviews.columns]
})
orderreviews_df_summary['dataset'] = 'order_reviews'
orderreviews_df_summary['null_value_total_pct'] = round(orderreviews_df_summary['null_value_total'] / orderreviews_df_summary['length'] * 100, 2)
orderreviews_df_summary= orderreviews_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
orderreviews_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,order_reviews,review_id,0,0.00,99173,100000,object
1,order_reviews,order_id,0,0.00,99441,100000,object
2,order_reviews,review_score,0,0.00,5,100000,int64
3,order_reviews,review_comment_title,88285,88.28,4600,100000,object
4,order_reviews,review_comment_message,58247,58.25,36921,100000,object
5,order_reviews,review_creation_date,0,0.00,637,100000,object
6,order_reviews,review_answer_timestamp,0,0.00,99010,100000,object


<div class="alert alert-block alert-success">
<b>Insight:</b>  
    Order Reviews memiliki 7 fitur yang nantinya akan dilakukan merging adalah fitur review_score.
</div>

# Orders Datasets

In [79]:
orders= pd.read_csv('./Datasets/olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',\
                                                            'order_delivered_customer_date', 'order_estimated_delivery_date'])
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [80]:
orders.describe()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,d2855109f77d3c3202bb4be96278e124,89b009b9871c7df3d23c56b673ddef57,delivered,2018-03-31 15:08:21,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-14 20:02:44,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522
first,NaN,NaN,NaN,2016-09-04 21:15:19,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-09-30 00:00:00
last,NaN,NaN,NaN,2018-10-17 17:30:18,2018-09-03 17:40:06,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-11-12 00:00:00


In [81]:
# Contoh Order Pelanggan
orders[orders['order_id'] == '00143d0f86d6fbd9f9b38ab440ac16f5']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
36265,00143d0f86d6fbd9f9b38ab440ac16f5,2e45292979b9b2700ea09560eeb0f803,delivered,2017-10-16 15:29:43,2017-10-16 16:07:52,2017-10-16 23:04:43,2017-10-27 18:43:56,2017-11-06


In [82]:
orders_df_summary = pd.DataFrame({
    'columns': orders.columns,
    'null_value_total': [orders[i].isnull().sum() for i in orders.columns],
    'unique_value_total': [orders[i].nunique() for i in orders.columns],
    'length': [orders[i].shape[0] for i in orders.columns],
    'data_type': [orders[i].dtypes for i in orders.columns]
})
orders_df_summary['dataset'] = 'orders'
orders_df_summary['null_value_total_pct'] = round(orders_df_summary['null_value_total'] / orders_df_summary['length'] * 100, 2)
orders_df_summary= orders_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
orders_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,orders,order_id,0,0.00,99441,99441,object
1,orders,customer_id,0,0.00,99441,99441,object
2,orders,order_status,0,0.00,8,99441,object
3,orders,order_purchase_timestamp,0,0.00,98875,99441,datetime64[ns]
4,orders,order_approved_at,160,0.16,90733,99441,datetime64[ns]
5,orders,order_delivered_carrier_date,1783,1.79,81018,99441,datetime64[ns]
6,orders,order_delivered_customer_date,2965,2.98,95664,99441,datetime64[ns]
7,orders,order_estimated_delivery_date,0,0.00,459,99441,datetime64[ns]


<div class="alert alert-block alert-success">
Dataset orders memiliki 8 kolom. Terdapat 3 kolom yang memiliki null_value yang menandakan ada barang yang belum diterima oleh customers dan juga ada barang yang belum dikirim kepada customers. Selanjutnya, akan dipakai hanya barang yang telah 'delivered' saja.
</div>

# Products

In [83]:
products = pd.read_csv('./Datasets/olist_products_dataset.csv')
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [84]:
products.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [85]:
products_df_summary = pd.DataFrame({
    'columns': products.columns,
    'null_value_total': [products[i].isnull().sum() for i in products.columns],
    'unique_value_total': [products[i].nunique() for i in products.columns],
    'length': [products[i].shape[0] for i in products.columns],
    'data_type': [products[i].dtypes for i in products.columns]
})
products_df_summary['dataset'] = 'products'
products_df_summary['null_value_total_pct'] = round(products_df_summary['null_value_total'] / products_df_summary['length'] * 100, 2)
products_df_summary= products_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
products_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,products,product_id,0,0.00,32951,32951,object
1,products,product_category_name,610,1.85,73,32951,object
2,products,product_name_lenght,610,1.85,66,32951,float64
3,products,product_description_lenght,610,1.85,2960,32951,float64
4,products,product_photos_qty,610,1.85,19,32951,float64
5,products,product_weight_g,2,0.01,2204,32951,float64
6,products,product_length_cm,2,0.01,99,32951,float64
7,products,product_height_cm,2,0.01,102,32951,float64
8,products,product_width_cm,2,0.01,95,32951,float64


<div class="alert alert-block alert-success">
Dataset products memiliki 9 kolom. Terdapat 3 kolom yang memiliki null value. Jumlah product_id adalah 32951 (produk yang terdapat pada e-commerce).
</div>

# Sellers

In [86]:
sellers = pd.read_csv('./Datasets/olist_sellers_dataset.csv', dtype={'seller_zip_code_prefix': str})
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [87]:
sellers_df_summary = pd.DataFrame({
    'columns': sellers.columns,
    'null_value_total': [sellers[i].isnull().sum() for i in sellers.columns],
    'unique_value_total': [sellers[i].nunique() for i in sellers.columns],
    'length': [sellers[i].shape[0] for i in sellers.columns],
    'data_type': [sellers[i].dtypes for i in sellers.columns]
})
sellers_df_summary['dataset'] = 'sellers'
sellers_df_summary['null_value_total_pct'] = round(sellers_df_summary['null_value_total'] / sellers_df_summary['length'] * 100, 2)
sellers_df_summary= sellers_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
sellers_df_summary

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,sellers,seller_id,0,0.0,3095,3095,object
1,sellers,seller_zip_code_prefix,0,0.0,2246,3095,object
2,sellers,seller_city,0,0.0,611,3095,object
3,sellers,seller_state,0,0.0,23,3095,object


# Product Category Name Translate

In [88]:
product_translate = pd.read_csv('./Datasets/product_category_name_translation.csv')
product_translate

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
5,esporte_lazer,sports_leisure
6,perfumaria,perfumery
7,utilidades_domesticas,housewares
8,telefonia,telephony
9,relogios_presentes,watches_gifts


In [89]:
product_translate_df_summary = pd.DataFrame({
    'columns': product_translate.columns,
    'null_value_total': [product_translate[i].isnull().sum() for i in product_translate.columns],
    'length': [product_translate[i].shape[0] for i in product_translate.columns],
    'data_type': [product_translate[i].dtypes for i in product_translate.columns]
})
product_translate_df_summary['dataset'] = 'product_translate'
product_translate_df_summary['null_value_total_pct'] = round(product_translate_df_summary['null_value_total'] / product_translate_df_summary['length'] * 100, 2)
product_translate_df_summary= product_translate_df_summary[['dataset', 'columns', 'null_value_total', 'null_value_total_pct', 'length', 'data_type']]
product_translate_df_summary

,dataset,columns,null_value_total,null_value_total_pct,length,data_type
0,product_translate,product_category_name,0,0.0,71,object
1,product_translate,product_category_name_english,0,0.0,71,object


In [90]:
product_translate.shape

(71, 2)

In [91]:
product_translate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [92]:
product_translate.describe()

,product_category_name,product_category_name_english
count,71,71
unique,71,71
top,construcao_ferramentas_ferramentas,office_furniture
freq,1,1


# Summary of Datasets

In [93]:
pd.concat([customers_df_summary, geolocation_df_summary, orderitems_df_summary, orderreviews_df_summary, orders_df_summary, products_df_summary, sellers_df_summary, product_translate_df_summary], ignore_index=True)

,dataset,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,customers,customer_id,0,0.00,99441.0,99441,object
1,customers,customer_unique_id,0,0.00,96096.0,99441,object
2,customers,customer_zip_code_prefix,0,0.00,14994.0,99441,object
3,customers,customer_city,0,0.00,4119.0,99441,object
4,customers,customer_state,0,0.00,27.0,99441,object
5,geolocation,geolocation_zip_code_prefix,0,0.00,19015.0,1000163,object
6,geolocation,geolocation_lat,0,0.00,717358.0,1000163,float64
7,geolocation,geolocation_lng,0,0.00,717613.0,1000163,float64
8,geolocation,geolocation_city,0,0.00,5971.0,1000163,object
9,geolocation,geolocation_state,0,0.00,27.0,1000163,object


# MERGING DATASETS

## <p style="color: brown">1) Datasets: customers.customer_id & orders.customer_id</p>

- Jumlah row masing-masing dataset orders dan customers adalah 99441. 

In [94]:
cust_and_order_df = pd.merge(orders, customers, how='outer', on='customer_id')
# Tidak memakai kolom-kolom berikut ini
cust_and_order_df.drop(columns=['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], inplace=True)
cust_and_order_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,72632f0f9dd73dfee390c9b22eb56dd6,09195,santo andre,SP


In [95]:
cust_and_order_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  99441 non-null  object        
 1   customer_id               99441 non-null  object        
 2   order_status              99441 non-null  object        
 3   order_purchase_timestamp  99441 non-null  datetime64[ns]
 4   customer_unique_id        99441 non-null  object        
 5   customer_zip_code_prefix  99441 non-null  object        
 6   customer_city             99441 non-null  object        
 7   customer_state            99441 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 6.8+ MB


## <p style="color: brown">2) Datasets: cust_and_order_df.order_id & order_items.order_id</p>

In [96]:
custorder_orderitems_df = pd.merge(cust_and_order_df, order_items, how='outer', on='order_id')
custorder_orderitems_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,72632f0f9dd73dfee390c9b22eb56dd6,09195,santo andre,SP,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,19.90,8.72


In [97]:
# Karena akan menganalisis order yang sudah dikirim saja 
# maka order_status akan dipilih yang 'delivered'
custorder_orderitems_df = custorder_orderitems_df[custorder_orderitems_df['order_status'] == 'delivered']
custorder_orderitems_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,72632f0f9dd73dfee390c9b22eb56dd6,09195,santo andre,SP,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,19.90,8.72


In [98]:
# Menghapus fitur order_status
custorder_orderitems_df.drop(columns='order_status', inplace=True)

In [99]:
# Contoh Pelanggan yang sekali transaksi membeli produk > 1
# dan membeli kembali produk di kemudian hari
custorder_orderitems_df[custorder_orderitems_df['customer_unique_id'] == 'c8460e4251689ba205045f3ea17884a1']

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value
42262,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,2018-08-07 09:03:02,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,1.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
42263,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,2018-08-07 09:03:02,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,2.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
42264,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,2018-08-07 09:03:02,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,3.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
42265,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,2018-08-07 09:03:02,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,4.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
42266,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,2018-08-07 09:03:02,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,5.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
42267,cb1f3a44e8b8527e16913306a4d3de2f,dbe9495069f4ddb6875dfc83462d616f,2018-08-07 09:03:02,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,6.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
49779,03aba68b07658f28f29612641f08d4ba,a7ab31829dc9a10f37e82b1e1afd26b6,2018-08-08 14:27:15,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,1.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
49780,03aba68b07658f28f29612641f08d4ba,a7ab31829dc9a10f37e82b1e1afd26b6,2018-08-08 14:27:15,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,2.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
49781,03aba68b07658f28f29612641f08d4ba,a7ab31829dc9a10f37e82b1e1afd26b6,2018-08-08 14:27:15,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,3.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44
49782,03aba68b07658f28f29612641f08d4ba,a7ab31829dc9a10f37e82b1e1afd26b6,2018-08-08 14:27:15,c8460e4251689ba205045f3ea17884a1,91170,porto alegre,RS,4.0,e7cc48a9daff5436f63d3aad9426f28b,53243585a1d6dc2643021fd1853d8905,170.0,30.44


In [100]:
custorder_orderitems_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110197 entries, 0 to 113424
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  110197 non-null  object        
 1   customer_id               110197 non-null  object        
 2   order_purchase_timestamp  110197 non-null  datetime64[ns]
 3   customer_unique_id        110197 non-null  object        
 4   customer_zip_code_prefix  110197 non-null  object        
 5   customer_city             110197 non-null  object        
 6   customer_state            110197 non-null  object        
 7   order_item_id             110197 non-null  float64       
 8   product_id                110197 non-null  object        
 9   seller_id                 110197 non-null  object        
 10  price                     110197 non-null  float64       
 11  freight_value             110197 non-null  float64       
dtypes:

In [101]:
custorder_orderitems_df[custorder_orderitems_df['order_id'] == '1b15974a0141d54e36626dca3fdc731a']

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value
99077,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,1.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99078,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,2.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99079,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,3.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99080,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,4.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99081,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,5.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99082,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,6.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99083,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,7.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99084,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,8.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99085,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,9.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12
99086,1b15974a0141d54e36626dca3fdc731a,be1b70680b9f9694d8c70f41fa3dc92b,2018-02-22 15:30:41,c402f431464c72e27330a67f7b94d4fb,03029,sao paulo,SP,10.0,ee3d532c8a438679776d222e997606b3,8e6d7754bc7e0f22c96d255ebda59eba,100.0,10.12


## <p style="color: brown">3) Datasets: custorder_orderitems_df.product_id & products.product_id</p>

In [102]:
custorder_orderitems_product_df = pd.merge(custorder_orderitems_df, products, how='outer', on='product_id')
custorder_orderitems_product_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0


In [103]:
custorder_orderitems_product_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110932 entries, 0 to 110931
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   order_id                    110197 non-null  object        
 1   customer_id                 110197 non-null  object        
 2   order_purchase_timestamp    110197 non-null  datetime64[ns]
 3   customer_unique_id          110197 non-null  object        
 4   customer_zip_code_prefix    110197 non-null  object        
 5   customer_city               110197 non-null  object        
 6   customer_state              110197 non-null  object        
 7   order_item_id               110197 non-null  float64       
 8   product_id                  110932 non-null  object        
 9   seller_id                   110197 non-null  object        
 10  price                       110197 non-null  float64       
 11  freight_value               110197 non-

## <p style="color: brown">4) Datasets: custorder_orderitems_product_df.product_category_name & product_translate.product_category_name</p>

In [104]:
custorder_orderitems_product_tr_df = pd.merge(custorder_orderitems_product_df, product_translate, how='outer', on='product_category_name')
custorder_orderitems_product_tr_df.drop(columns='product_category_name', inplace=True)
custorder_orderitems_product_tr_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares
4,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,2017-11-24 21:27:48,3e4fd73f1e86b135b9b121d6abbe9597,19400,presidente venceslau,SP,1.0,be021417a6acb56b9b50d3fd2714baa8,f5f46307a4d15880ca14fab4ad9dfc9b,339.00,17.12,48.0,664.0,6.0,14300.0,38.0,34.0,34.0,housewares


## <p style="color: brown">5) Datasets: custorder_orderitems_product_tr_df.customer_zip_code_prefix & temp_geo.customer_zip_code_prefix</p>

In [105]:
custorder_orderitems_product_tr_temp_geo_df = pd.merge(custorder_orderitems_product_tr_df, temp_geo, how='outer', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')
custorder_orderitems_product_tr_temp_geo_df.drop(columns='geolocation_zip_code_prefix', inplace=True)
custorder_orderitems_product_tr_temp_geo_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,geolocation_lat,geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161
1,70b35acffdf851e782ebf6fbc35eb620,8e8ee9b08afb49b080d193f98b0505af,2018-03-22 17:23:21,8a4002923e801e3120a11070fd31c9e2,03149,sao paulo,SP,1.0,6cc44821f36f3156c782da72dd634e47,da8622b14eb17ae2831f4ac5b9dab84a,99.90,11.79,55.0,273.0,1.0,1050.0,38.0,10.0,38.0,bed_bath_table,-23.576983,-46.587161
2,70b35acffdf851e782ebf6fbc35eb620,8e8ee9b08afb49b080d193f98b0505af,2018-03-22 17:23:21,8a4002923e801e3120a11070fd31c9e2,03149,sao paulo,SP,2.0,6cc44821f36f3156c782da72dd634e47,da8622b14eb17ae2831f4ac5b9dab84a,99.90,11.79,55.0,273.0,1.0,1050.0,38.0,10.0,38.0,bed_bath_table,-23.576983,-46.587161
3,69923a4e07ce446644394df37a710286,31f31efcb333fcbad2b1371c8cf0fa84,2017-09-04 11:26:38,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,9abb00920aae319ef9eba674b7d2e6ff,1771297ac436903d1dd6b0e9279aa505,35.39,8.72,35.0,2395.0,1.0,350.0,19.0,14.0,12.0,baby,-23.576983,-46.587161
4,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268


In [106]:
custorder_orderitems_product_tr_temp_geo_df.rename(columns={'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'}, inplace=True)

In [107]:
custorder_orderitems_product_tr_temp_geo_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161
1,70b35acffdf851e782ebf6fbc35eb620,8e8ee9b08afb49b080d193f98b0505af,2018-03-22 17:23:21,8a4002923e801e3120a11070fd31c9e2,03149,sao paulo,SP,1.0,6cc44821f36f3156c782da72dd634e47,da8622b14eb17ae2831f4ac5b9dab84a,99.90,11.79,55.0,273.0,1.0,1050.0,38.0,10.0,38.0,bed_bath_table,-23.576983,-46.587161
2,70b35acffdf851e782ebf6fbc35eb620,8e8ee9b08afb49b080d193f98b0505af,2018-03-22 17:23:21,8a4002923e801e3120a11070fd31c9e2,03149,sao paulo,SP,2.0,6cc44821f36f3156c782da72dd634e47,da8622b14eb17ae2831f4ac5b9dab84a,99.90,11.79,55.0,273.0,1.0,1050.0,38.0,10.0,38.0,bed_bath_table,-23.576983,-46.587161
3,69923a4e07ce446644394df37a710286,31f31efcb333fcbad2b1371c8cf0fa84,2017-09-04 11:26:38,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,9abb00920aae319ef9eba674b7d2e6ff,1771297ac436903d1dd6b0e9279aa505,35.39,8.72,35.0,2395.0,1.0,350.0,19.0,14.0,12.0,baby,-23.576983,-46.587161
4,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268


In [108]:
custorder_orderitems_product_tr_temp_geo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115210 entries, 0 to 115209
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       110197 non-null  object        
 1   customer_id                    110197 non-null  object        
 2   order_purchase_timestamp       110197 non-null  datetime64[ns]
 3   customer_unique_id             110197 non-null  object        
 4   customer_zip_code_prefix       110197 non-null  object        
 5   customer_city                  110197 non-null  object        
 6   customer_state                 110197 non-null  object        
 7   order_item_id                  110197 non-null  float64       
 8   product_id                     110932 non-null  object        
 9   seller_id                      110197 non-null  object        
 10  price                          110197 non-null  float64       
 11  

## <p style="color: brown">6) Datasets: custorder_orderitems_product_tr_temp_geo_df.order_id & order_review.order_id</p>

In [109]:
custorder_orderitems_product_tr_temp_geo_orderreviewsdf = pd.merge(custorder_orderitems_product_tr_temp_geo_df, order_reviews, how='outer', on='order_id')
custorder_orderitems_product_tr_temp_geo_orderreviewsdf.drop(columns=['review_answer_timestamp', 'review_creation_date', 'review_comment_message', 'review_comment_title', 'review_id'], inplace=True)
custorder_orderitems_product_tr_temp_geo_orderreviewsdf.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0
1,70b35acffdf851e782ebf6fbc35eb620,8e8ee9b08afb49b080d193f98b0505af,2018-03-22 17:23:21,8a4002923e801e3120a11070fd31c9e2,03149,sao paulo,SP,1.0,6cc44821f36f3156c782da72dd634e47,da8622b14eb17ae2831f4ac5b9dab84a,99.90,11.79,55.0,273.0,1.0,1050.0,38.0,10.0,38.0,bed_bath_table,-23.576983,-46.587161,5.0
2,70b35acffdf851e782ebf6fbc35eb620,8e8ee9b08afb49b080d193f98b0505af,2018-03-22 17:23:21,8a4002923e801e3120a11070fd31c9e2,03149,sao paulo,SP,2.0,6cc44821f36f3156c782da72dd634e47,da8622b14eb17ae2831f4ac5b9dab84a,99.90,11.79,55.0,273.0,1.0,1050.0,38.0,10.0,38.0,bed_bath_table,-23.576983,-46.587161,5.0
3,69923a4e07ce446644394df37a710286,31f31efcb333fcbad2b1371c8cf0fa84,2017-09-04 11:26:38,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,9abb00920aae319ef9eba674b7d2e6ff,1771297ac436903d1dd6b0e9279aa505,35.39,8.72,35.0,2395.0,1.0,350.0,19.0,14.0,12.0,baby,-23.576983,-46.587161,5.0
4,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0


In [110]:
custorder_orderitems_product_tr_temp_geo_orderreviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118846 entries, 0 to 118845
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113833 non-null  object        
 1   customer_id                    110848 non-null  object        
 2   order_purchase_timestamp       110848 non-null  datetime64[ns]
 3   customer_unique_id             110848 non-null  object        
 4   customer_zip_code_prefix       110848 non-null  object        
 5   customer_city                  110848 non-null  object        
 6   customer_state                 110848 non-null  object        
 7   order_item_id                  110848 non-null  float64       
 8   product_id                     111583 non-null  object        
 9   seller_id                      110848 non-null  object        
 10  price                          110848 non-null  float64       
 11  

## <p style="color: brown">7) Datasets: custorder_orderitems_product_tr_temp_geo_orderreviewsdf_df.seller_id & sellers.seller_id</p>

In [111]:
custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df = pd.merge(custorder_orderitems_product_tr_temp_geo_orderreviewsdf, sellers, how='outer', on='seller_id')
custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0,09350,maua,SP
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0,09350,maua,SP
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.600462,-46.655318,5.0,09350,maua,SP
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-27.528367,-48.491914,3.0,09350,maua,SP
4,61dc7d82d2eb6ecf4f0ac2a3cd693cfb,eda669d85ef713eb385cf57a584a2b68,2017-11-05 21:54:53,cb85e26bc7feed69b59cdd1bc1aecb86,04213,sao paulo,SP,1.0,03b58043df5d7424df7eac2be2511c51,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1650.0,30.0,4.0,30.0,cool_stuff,-23.592504,-46.602403,5.0,09350,maua,SP


In [112]:
temp_geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,01001,-23.550190,-46.634024
1,01002,-23.548146,-46.634979
2,01003,-23.548994,-46.635731
3,01004,-23.549799,-46.634757
4,01005,-23.549456,-46.636733


In [113]:
custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df = pd.merge(custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df, temp_geo, how='outer',left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')
custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0,09350,maua,SP,09350,-23.680729,-46.444238
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0,09350,maua,SP,09350,-23.680729,-46.444238
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.600462,-46.655318,5.0,09350,maua,SP,09350,-23.680729,-46.444238
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-27.528367,-48.491914,3.0,09350,maua,SP,09350,-23.680729,-46.444238
4,61dc7d82d2eb6ecf4f0ac2a3cd693cfb,eda669d85ef713eb385cf57a584a2b68,2017-11-05 21:54:53,cb85e26bc7feed69b59cdd1bc1aecb86,04213,sao paulo,SP,1.0,03b58043df5d7424df7eac2be2511c51,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1650.0,30.0,4.0,30.0,cool_stuff,-23.592504,-46.602403,5.0,09350,maua,SP,09350,-23.680729,-46.444238


In [114]:
custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df.rename({'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'}, inplace=True)

In [115]:
custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0,09350,maua,SP,09350,-23.680729,-46.444238
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0,09350,maua,SP,09350,-23.680729,-46.444238
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.600462,-46.655318,5.0,09350,maua,SP,09350,-23.680729,-46.444238
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-27.528367,-48.491914,3.0,09350,maua,SP,09350,-23.680729,-46.444238
4,61dc7d82d2eb6ecf4f0ac2a3cd693cfb,eda669d85ef713eb385cf57a584a2b68,2017-11-05 21:54:53,cb85e26bc7feed69b59cdd1bc1aecb86,04213,sao paulo,SP,1.0,03b58043df5d7424df7eac2be2511c51,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1650.0,30.0,4.0,30.0,cool_stuff,-23.592504,-46.602403,5.0,09350,maua,SP,09350,-23.680729,-46.444238


# Memindahkan data ke dataset baru

In [116]:
df = custorder_orderitems_product_tr_temp_geo_orderreviews_sellers_df.copy()
df.head(10)

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0,09350,maua,SP,09350,-23.680729,-46.444238
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0,09350,maua,SP,09350,-23.680729,-46.444238
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.600462,-46.655318,5.0,09350,maua,SP,09350,-23.680729,-46.444238
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-27.528367,-48.491914,3.0,09350,maua,SP,09350,-23.680729,-46.444238
4,61dc7d82d2eb6ecf4f0ac2a3cd693cfb,eda669d85ef713eb385cf57a584a2b68,2017-11-05 21:54:53,cb85e26bc7feed69b59cdd1bc1aecb86,04213,sao paulo,SP,1.0,03b58043df5d7424df7eac2be2511c51,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1650.0,30.0,4.0,30.0,cool_stuff,-23.592504,-46.602403,5.0,09350,maua,SP,09350,-23.680729,-46.444238
5,a0151737f2f0c6c0a5fd69d45f66ceea,fc2697314ab7fbeda62bb6f1afa4efcd,2017-07-12 14:11:58,bdd30e6e39cc70dde8665187b57af402,13820,jaguariuna,SP,1.0,725cbfcaff95a4d43742fdf13cf43c75,3504c0cb71d7fa48d967e0e4c94d59d9,44.99,7.78,58.0,537.0,2.0,1850.0,34.0,5.0,34.0,cool_stuff,-22.692949,-46.982878,4.0,09350,maua,SP,09350,-23.680729,-46.444238
6,0010b2e5201cc5f1ae7e9c6cc8f5bd00,57ef317d4818cb42680fc9dfd13867ce,2017-09-11 17:39:33,e883f26917e2095c439f70c2cf8c914b,22461,rio de janeiro,RJ,1.0,5a419dbf24a8c9718fe522b81c69f61a,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,16.60,53.0,537.0,2.0,1800.0,34.0,5.0,34.0,cool_stuff,-22.961345,-43.213641,5.0,09350,maua,SP,09350,-23.680729,-46.444238
7,a30b65719297f5afcb931bcad170cb7d,d2d82b8e87143477f9a27e005f4b87f3,2017-08-31 17:49:40,3c601cdf47f343edf04dbd987a837a2a,11250,bertioga,SP,1.0,c6d0daec0037d0ac4ee036ddff197b35,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,13.37,56.0,645.0,2.0,1800.0,34.0,4.0,34.0,cool_stuff,-23.818390,-46.080624,5.0,09350,maua,SP,09350,-23.680729,-46.444238
8,bea597ea1f5ff4fa02bcaf403531e944,4a2f70b7ed35b1881bab73d2be8d440c,2017-10-20 22:34:48,9a6d3775ed3c4dc33243faf14264f1a9,12941,atibaia,SP,1.0,c6d0daec0037d0ac4ee036ddff197b35,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1800.0,34.0,4.0,34.0,cool_stuff,-23.123122,-46.541581,5.0,09350,maua,SP,09350,-23.680729,-46.444238
9,0d9a4836459b4ededef3b8dfd7978d3b,ca6a166e2e426cd1120c7929a257408f,2017-07-24 11:20:43,4c0a51f04c6f491f080bcdf3baf7eff8,11920,iguape,SP,1.0,5a419dbf24a8c9718fe522b81c69f61a,3504c0cb71d7fa48d967e0e4c94d59d9,44.49,11.85,53.0,537.0,2.0,1800.0,34.0,5.0,34.0,cool_stuff,-24.704620,-47.555360,5.0,09350,maua,SP,09350,-23.680729,-46.444238


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135747 entries, 0 to 135746
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113833 non-null  object        
 1   customer_id                    110848 non-null  object        
 2   order_purchase_timestamp       110848 non-null  datetime64[ns]
 3   customer_unique_id             110848 non-null  object        
 4   customer_zip_code_prefix       110848 non-null  object        
 5   customer_city                  110848 non-null  object        
 6   customer_state                 110848 non-null  object        
 7   order_item_id                  110848 non-null  float64       
 8   product_id                     111583 non-null  object        
 9   seller_id                      110973 non-null  object        
 10  price                          110848 non-null  float64       
 11  

In [118]:
df['order_date_day'] = df['order_purchase_timestamp'].dt.day
df['order_date_day_name'] = df['order_purchase_timestamp'].dt.day_name()
df['order_date_month'] = df['order_purchase_timestamp'].dt.month
df['order_date_month_name'] = df['order_purchase_timestamp'].dt.month_name()
df['order_date_year'] = df['order_purchase_timestamp'].dt.year
df['order_date_monthyear'] = df['order_purchase_timestamp'].dt.to_period('M')
df['order_date_hour'] = df['order_purchase_timestamp'].dt.hour

# Semesters
def range_semester_date(x):
    '''
    Membagi purchase date dalam semester
    '''
    rentang = pd.date_range(start=df['order_purchase_timestamp'].min(), periods=6, freq='6M')
    x = pd.to_datetime(x).date()
    if x >= rentang[0] and x <= rentang[1]:
        return "1"
    elif x > rentang[1] and x <= rentang[2]:
        return "2"
    elif x > rentang[2] and x <= rentang[3]:
        return "3"
    elif x > rentang[3] and x <= rentang[4]:
        return "4"
    elif x > rentang[4] and x <= rentang[5]:
        return "5"
    elif x > rentang[5] and x <= rentang[6]:
        return "6"
    
df['semester'] = df['order_purchase_timestamp'].apply(lambda x: range_semester_date(x))
df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,order_date_day,order_date_day_name,order_date_month,order_date_month_name,order_date_year,order_date_monthyear,order_date_hour,semester
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0,09350,maua,SP,09350,-23.680729,-46.444238,2.0,Monday,10.0,October,2017.0,2017-10,10.0,3
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0,09350,maua,SP,09350,-23.680729,-46.444238,15.0,Tuesday,8.0,August,2017.0,2017-08,18.0,2
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.600462,-46.655318,5.0,09350,maua,SP,09350,-23.680729,-46.444238,2.0,Wednesday,8.0,August,2017.0,2017-08,18.0,2
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-27.528367,-48.491914,3.0,09350,maua,SP,09350,-23.680729,-46.444238,23.0,Monday,10.0,October,2017.0,2017-10,23.0,3
4,61dc7d82d2eb6ecf4f0ac2a3cd693cfb,eda669d85ef713eb385cf57a584a2b68,2017-11-05 21:54:53,cb85e26bc7feed69b59cdd1bc1aecb86,04213,sao paulo,SP,1.0,03b58043df5d7424df7eac2be2511c51,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1650.0,30.0,4.0,30.0,cool_stuff,-23.592504,-46.602403,5.0,09350,maua,SP,09350,-23.680729,-46.444238,5.0,Sunday,11.0,November,2017.0,2017-11,21.0,3


In [119]:
df.head()

,order_id,customer_id,order_purchase_timestamp,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_lat,customer_lng,review_score,seller_zip_code_prefix,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,order_date_day,order_date_day_name,order_date_month,order_date_month_name,order_date_year,order_date_monthyear,order_date_hour,semester
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff,03149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.576983,-46.587161,4.0,09350,maua,SP,09350,-23.680729,-46.444238,2.0,Monday,10.0,October,2017.0,2017-10,10.0,3
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,2017-08-15 18:29:31,3a51803cc0d012c3b5dc8b7528cb05f7,03366,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.564630,-46.534268,4.0,09350,maua,SP,09350,-23.680729,-46.444238,15.0,Tuesday,8.0,August,2017.0,2017-08,18.0,2
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,2017-08-02 18:24:47,ef0996a1a279c26e7ecbd737be23d235,02290,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-23.600462,-46.655318,5.0,09350,maua,SP,09350,-23.680729,-46.444238,2.0,Wednesday,8.0,August,2017.0,2017-08,18.0,2
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,2017-10-23 23:26:46,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0,housewares,-27.528367,-48.491914,3.0,09350,maua,SP,09350,-23.680729,-46.444238,23.0,Monday,10.0,October,2017.0,2017-10,23.0,3
4,61dc7d82d2eb6ecf4f0ac2a3cd693cfb,eda669d85ef713eb385cf57a584a2b68,2017-11-05 21:54:53,cb85e26bc7feed69b59cdd1bc1aecb86,04213,sao paulo,SP,1.0,03b58043df5d7424df7eac2be2511c51,3504c0cb71d7fa48d967e0e4c94d59d9,48.90,11.73,56.0,645.0,2.0,1650.0,30.0,4.0,30.0,cool_stuff,-23.592504,-46.602403,5.0,09350,maua,SP,09350,-23.680729,-46.444238,5.0,Sunday,11.0,November,2017.0,2017-11,21.0,3


In [120]:
df_summary = pd.DataFrame({
    'columns': df.columns,
    'null_value_total': [df[i].isnull().sum() for i in df.columns],
    'unique_value_total': [df[i].nunique() for i in df.columns],
    'length': [df[i].shape[0] for i in df.columns],
    'data_type': [df[i].dtypes for i in df.columns]
})
# df_summary['dataset'] = 'order_items'
df_summary['null_value_total_pct'] = round(df_summary['null_value_total'] / df_summary['length'] * 100, 2)
df_summary= df_summary[['columns', 'null_value_total', 'null_value_total_pct', 'unique_value_total', 'length', 'data_type']]
df_summary

,columns,null_value_total,null_value_total_pct,unique_value_total,length,data_type
0,order_id,21914,16.14,99441,135747,object
1,customer_id,24899,18.34,96478,135747,object
2,order_purchase_timestamp,24899,18.34,95956,135747,datetime64[ns]
3,customer_unique_id,24899,18.34,93358,135747,object
4,customer_zip_code_prefix,24899,18.34,14889,135747,object
5,customer_city,24899,18.34,4085,135747,object
6,customer_state,24899,18.34,27,135747,object
7,order_item_id,24899,18.34,21,135747,float64
8,product_id,24164,17.80,32951,135747,object
9,seller_id,24774,18.25,3095,135747,object


# Membuat Dataset Baru

In [121]:
df.to_csv('./Datasets/olist_dataset_all.csv')